In [ ]:
import os
import sys
import torch
import transformers
from datasets import load_dataset
from peft import (
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict,
    prepare_model_for_int8_training,
    set_peft_model_state_dict,
    PeftType,
    TaskType,
    PeftModelForSequenceClassification
)
from transformers import LlamaTokenizer ,LlamaForSequenceClassification ,LlamaForCausalLM,Trainer,TrainingArguments
import evaluate
import numpy as np

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
base_model = "../ljcllama-7b-hf"
device_map = "auto"
num_epochs = 20
lr = 5e-5
peft_type = PeftType.LORA
datasets = load_dataset("csv", data_files={"train":"train.csv","test":"valid.csv"})

In [ ]:

model = LlamaForSequenceClassification.from_pretrained(
        base_model,
        load_in_8bit = True,
        torch_dtype = torch.float16,
        num_labels = 66,
        device_map = device_map
    )

tokenizer = LlamaTokenizer.from_pretrained(base_model)
tokenizer.pad_token = -100
model.resize_token_embeddings(len(tokenizer))

In [ ]:
def compute_metrics(eval_pred):
        metirc = evaluate.load("accuracy")
        logits , labels = eval_pred
        predictions = np.argmax(logits,axis=-1)
        return metirc.compute(predictions=predictions,references=labels)


def tokenize_function(examples):
        return tokenizer(examples["text"],padding = "max_length",truncation = True) 

In [ ]:
model = prepare_model_for_int8_training(model)

config = LoraConfig(
        r=8,
        lora_alpha=16,
        inference_mode=False,
        lora_dropout=0.1,
        target_modules =  [
        "q_proj",
        "v_proj",
    ],
        task_type=TaskType.SEQ_CLS,
    )

model = get_peft_model(model, config)
model.model.score = model.model.score.float()

In [ ]:
model.print_trainable_parameters()
model

In [ ]:
tokenized_dataset = datasets.map(tokenize_function,batched=True)
small_train_dataset = tokenized_dataset["train"].shuffle(seed=42)
small_eval_dataset = tokenized_dataset["test"].shuffle(seed=42)
train_args = TrainingArguments(logging_strategy="steps",logging_steps=1,fp16=True,per_device_train_batch_size=2,eval_steps=10,evaluation_strategy="steps",per_device_eval_batch_size=1,num_train_epochs=num_epochs,learning_rate=lr)

In [ ]:
trainer = Trainer(model = model,args = train_args,train_dataset=small_train_dataset,eval_dataset=small_eval_dataset,compute_metrics=compute_metrics)
trainer.train()


In [ ]:
model.save_pretrained("llama-for-attack-ada")
evaldataset = tokenized_dataset['test'].remove_columns('text').rename_column('label','labels')

In [ ]:
model

In [ ]:
from torch.utils.data import DataLoader

def col_fn(examples):
    return tokenizer.pad(examples,padding="max_length",return_tensors="pt")
evaldataset = DataLoader(evaldataset,collate_fn=col_fn,batch_size=1)

In [ ]:
from tqdm import tqdm
from torch.functional import F
count = 0
model.eval()
#model.to(device)
for step,batch in enumerate(tqdm(evaldataset)):
    batch.to(device)
    with torch.no_grad():
        output = model(**batch)
    count += int(batch['labels']== torch.argmax(F.softmax(output.logits,dim=1),dim=1))
print(count/len(evaldataset))


In [ ]:
count

In [ ]:
model.save_pretrained("ljc-llama-for-attack-adaption")

In [ ]:
model  = LlamaForSequenceClassification.from_pretrained("ljcllama-7b-hf",num_labels = 66,return_dict=True,device_map="auto")

In [ ]:
model = PeftModelForSequenceClassification.from_pretrained(model,"ljcoutputdir")

In [ ]:
model.eval()